In [24]:
import pandas as pd
data = pd.read_csv('heart_processed.csv')

In [25]:
import pandas as pd

cols = [col for col in data.columns.tolist() if 'val' in col]

data=data.groupby(['cause', 'age', 'Year'])[cols].sum().reset_index()
data = data[data['Year'] > 2000]
data.sample(10)

,cause,age,Year,valprevnumberboth,valdeathsnumberboth,valprevrateboth,valdeathsrateboth,valprevpercentboth,valdeathspercentboth,valprevnumberfemale,...,valprevratefemale,valdeathsratefemale,valprevpercentfemale,valdeathspercentfemale,valprevnumbermale,valdeathsnumbermale,valprevratemale,valdeathsratemale,valprevpercentmale,valdeathspercentmale
886,Pulmonary Arterial Hypertension,15-49 years,2004,4.247700e+04,2.034480e+03,3.660196e+02,19.3078,0.0000,0.1395,2.578864e+04,...,4.564301e+02,22.4768,0.0002,0.2627,1.668836e+04,8.483246e+02,2.761946e+02,16.2725,0.0000,0.0851
883,Pulmonary Arterial Hypertension,15-49 years,2001,4.055620e+04,1.992087e+03,3.632231e+02,20.1705,0.0000,0.1374,2.468307e+04,...,4.532005e+02,23.5047,0.0002,0.2606,1.587313e+04,8.426643e+02,2.734432e+02,16.9783,0.0000,0.0840
478,Lower extremity peripheral arterial disease,15-49 years,2016,0.000000e+00,0.000000e+00,0.000000e+00,0.0000,0.0000,0.0000,0.000000e+00,...,0.000000e+00,0.0000,0.0000,0.0000,0.000000e+00,0.000000e+00,0.000000e+00,0.0000,0.0000,0.0000
353,Ischemic heart disease,75+ years,2007,4.727726e+07,3.487789e+06,5.470886e+06,416860.3238,54.7105,42.7393,2.368389e+07,...,4.683201e+06,397214.6754,46.8329,43.0235,2.359337e+07,1.522430e+06,6.526319e+06,452311.4422,65.2642,42.2465
560,Lower extremity peripheral arterial disease,75+ years,2014,3.265456e+07,4.533092e+04,2.314602e+06,3914.5537,23.1466,0.4554,2.336398e+07,...,2.690340e+06,3723.6179,26.9038,0.4669,9.290584e+06,1.836719e+04,1.780965e+06,4296.1477,17.8111,0.4436
396,Ischemic heart disease,Age-standardized,2008,0.000000e+00,0.000000e+00,6.107488e+05,30878.2533,6.3366,0.0000,0.000000e+00,...,4.752941e+05,25627.7891,4.8836,0.0000,0.000000e+00,0.000000e+00,7.616242e+05,37734.7325,7.9792,0.0000
409,Ischemic heart disease,Age-standardized,2021,0.000000e+00,0.000000e+00,6.218404e+05,26104.5277,6.4933,0.0000,0.000000e+00,...,4.898596e+05,21757.9296,5.0555,0.0000,0.000000e+00,0.000000e+00,7.672912e+05,31893.8253,8.1057,0.0000
152,Cardiovascular diseases,75+ years,2006,9.749067e+07,7.608186e+06,1.071166e+07,932855.9675,107.1192,93.4316,5.530081e+07,...,1.002925e+07,929675.1387,100.2943,98.7092,4.218986e+07,3.222175e+06,1.162294e+07,954823.7521,116.2320,86.9289
81,Cardiovascular diseases,15-49 years,2019,5.543276e+07,4.508867e+05,4.093609e+05,3129.7873,4.1587,22.0547,2.885736e+07,...,4.254114e+05,2182.2698,4.2787,21.7483,2.657540e+07,2.944787e+05,3.932821e+05,4039.4070,4.0338,22.3257
161,Cardiovascular diseases,75+ years,2015,1.299246e+08,9.384892e+06,1.078715e+07,874742.2697,107.8746,90.8736,7.232057e+07,...,1.010690e+07,880818.3529,101.0711,95.6742,5.760401e+07,4.175872e+06,1.168056e+07,882721.1192,116.8093,84.8568


In [26]:
data[data['Year']].sample(10)


KeyError: "None of [Index([2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,\n       ...\n       2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021],\n      dtype='int64', length=525)] are in the [columns]"

In [27]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
import numpy as np

def optimal_poly_fit(years, values, max_degree=4):
    """Find optimal polynomial degree using BIC"""
    bic_values = []
    models = []

    for degree in range(1, max_degree+1):
        # Create polynomial features
        poly = PolynomialFeatures(degree)
        X_poly = poly.fit_transform(years.reshape(-1, 1))

        # Fit model
        model = LinearRegression().fit(X_poly, values)
        pred = model.predict(X_poly)

        # Calculate BIC
        n = len(values)
        mse = np.mean((values - pred)**2)
        bic = n * np.log(mse) + degree * np.log(n)
        bic_values.append(bic)
        models.append(model)

    # Select degree with lowest BIC
    best_degree = np.argmin(bic_values) + 1
    return models[best_degree-1], best_degree

def extrapolate_group(group, val_cols=cols):  # Replace with actual columns
    """Extrapolate all value columns for a group"""
    group = group.sort_values('Year')
    if len(group) < 4:
        return group

    last_year = group['Year'].max()
    if last_year >= 2030:
        return group

    # Initialize DataFrame for new predictions
    future_years = np.arange(last_year + 1, 2031)
    new_rows = pd.DataFrame({
        'Year': future_years,
        'cause': group['cause'].iloc[0],
        'age': group['age'].iloc[0],
        # 'Location': group['Location'].mode()[0]
    })

    # Process each value column
    for col in val_cols:
        # Extract historical data
        years = group['Year'].values
        values = group[col].values

        # Fit optimal polynomial model
        model, degree = optimal_poly_fit(years, values)

        # Predict future values (including last year for blending)
        predict_years = np.arange(last_year, 2031)
        X_future = PolynomialFeatures(degree).fit_transform(predict_years.reshape(-1, 1))
        predictions = model.predict(X_future)

        # Store predictions (excluding last historical year)
        new_rows[col] = predictions[1:]

        # Blend with historical data
        if len(group) >= 3:
            hist_vals = group[col].iloc[-3:].values
            pred_vals = new_rows[col].iloc[:3].values
            blended = hist_vals * np.array([0.7, 0.3, 0.1]) + pred_vals * np.array([0.3, 0.7, 0.9])
            new_rows.loc[:2, col] = blended

    return pd.concat([group, new_rows], ignore_index=True)

# Apply to each group
final_data = (
    data.groupby(['cause', 'age'])
    .apply(extrapolate_group)
    .reset_index(drop=True)
)

/var/folders/7p/m0f5r_5s72g21m2zr560q7tc0000gn/T/ipykernel_13640/1673979104.py:23: RuntimeWarning: divide by zero encountered in log
  bic = n * np.log(mse) + degree * np.log(n)
/var/folders/7p/m0f5r_5s72g21m2zr560q7tc0000gn/T/ipykernel_13640/1673979104.py:23: RuntimeWarning: divide by zero encountered in log
  bic = n * np.log(mse) + degree * np.log(n)
/var/folders/7p/m0f5r_5s72g21m2zr560q7tc0000gn/T/ipykernel_13640/1673979104.py:23: RuntimeWarning: divide by zero encountered in log
  bic = n * np.log(mse) + degree * np.log(n)
/var/folders/7p/m0f5r_5s72g21m2zr560q7tc0000gn/T/ipykernel_13640/1673979104.py:23: RuntimeWarning: divide by zero encountered in log
  bic = n * np.log(mse) + degree * np.log(n)
/var/folders/7p/m0f5r_5s72g21m2zr560q7tc0000gn/T/ipykernel_13640/1673979104.py:23: RuntimeWarning: divide by zero encountered in log
  bic = n * np.log(mse) + degree * np.log(n)
/var/folders/7p/m0f5r_5s72g21m2zr560q7tc0000gn/T/ipykernel_13640/1673979104.py:23: RuntimeWarning: divide by z

In [29]:
# In your data preprocessing script
final_data['is_projection'] = final_data['Year'] > 2021
final_data.head()

,cause,age,Year,valprevnumberboth,valdeathsnumberboth,valprevrateboth,valdeathsrateboth,valprevpercentboth,valdeathspercentboth,valprevnumberfemale,...,valdeathsratefemale,valprevpercentfemale,valdeathspercentfemale,valprevnumbermale,valdeathsnumbermale,valprevratemale,valdeathsratemale,valprevpercentmale,valdeathspercentmale,is_projection
0,Cardiovascular diseases,0-14 years,2001,1.006033e+07,95294.2033,131648.8974,917.9349,1.4317,3.1463,4.704098e+06,...,905.4473,1.3642,3.4143,5.356233e+06,49859.0422,136053.5001,930.6520,1.5005,2.9246,False
1,Cardiovascular diseases,0-14 years,2002,1.015222e+07,91550.4596,132041.6146,889.3312,1.4382,3.1377,4.742296e+06,...,875.8339,1.3702,3.3922,5.409927e+06,47875.0452,136375.5019,902.9746,1.5065,2.9279,False
2,Cardiovascular diseases,0-14 years,2003,1.024996e+07,88215.8490,132434.7553,861.1266,1.4446,3.1288,4.783663e+06,...,847.6343,1.3764,3.3763,5.466294e+06,46206.9692,136693.2950,874.6903,1.5133,2.9259,False
3,Cardiovascular diseases,0-14 years,2004,1.033748e+07,85766.0801,132737.0986,833.7668,1.4506,3.0785,4.821710e+06,...,819.9156,1.3826,3.3201,5.515765e+06,45094.6327,136923.3694,847.6490,1.5181,2.8792,False
4,Cardiovascular diseases,0-14 years,2005,1.040483e+07,84325.8046,132862.4102,811.9857,1.4535,3.1149,4.852577e+06,...,796.7006,1.3866,3.3602,5.552255e+06,44429.0491,136986.7338,827.2624,1.5214,2.9130,False


In [30]:
final_data.to_csv('trends.csv')